In [ ]:
import numpy as np

In [ ]:
TILE_DIM = 4
gridDim = 2
BLOCK_ROWS = 1

def transpose(mat, N, thParams, tile_ur, tile_ll):
    threadId, blockId_x, blockId_y = thParams
    A = mat.copy()
    A = A.ravel()

    x = blockId_x * TILE_DIM
    y = blockId_y * TILE_DIM

    is_ondiag = True if x==y else False
    print(f"ondiag: {is_ondiag}")

    if is_ondiag:
        j = 0
        while(j < TILE_DIM):
            tile_ur[j, threadId] = A[(y + threadId)*N + j + x]
            j += BLOCK_ROWS

    else:
        j = 0
        while(j < TILE_DIM):
            tile_ur[j, threadId] = A[(y + threadId)*N + j + x]
            tile_ll[j, threadId] = A[(x + threadId)*N + j + y]
            j += BLOCK_ROWS
        

def transpose_enplace(mat, N, thParams, tile_ur, tile_ll):
    threadId, blockId_x, blockId_y = thParams
    A = mat.copy()
    A = A.ravel()

    x = blockId_x * TILE_DIM
    y = blockId_y * TILE_DIM

    ur_cp = tile_ur.copy()
    ur_cp = ur_cp.ravel()

    is_ondiag = True if x==y else False
    if is_ondiag:
        j = 0
        while(j < TILE_DIM):
            #A[(y + threadId)*N + j + x] = tile_ur[threadId, j]
            A[(y + threadId)*N + j + x] = ur_cp[threadId*TILE_DIM + j]
            j += BLOCK_ROWS

    else:
        ll_cp = tile_ll.copy()
        ll_cp = ll_cp.ravel()

        j = 0
        while(j < TILE_DIM):
            A[(y + threadId)*N + j + x] = ll_cp[threadId*TILE_DIM + j]
            A[(x + threadId)*N + j + y] = ur_cp[threadId*TILE_DIM + j]
            j += BLOCK_ROWS
        
    return A.reshape((N, N))

In [ ]:
N = 8
A = np.array([i for i in range(N*N)])
A = A.reshape((N,N))

AT = A.transpose()

display(A)
for blockId_x, blockId_y in [(0,0), (1,0), (1,1)]:
    print(f"Block: ({blockId_x}, {blockId_y})")
    tile_ur = np.zeros((TILE_DIM,TILE_DIM))
    tile_ll = np.zeros((TILE_DIM,TILE_DIM))
    for i in range(4):
        threadId = i
        thParams = threadId, blockId_x, blockId_y 
        transpose(A, N, thParams, tile_ur, tile_ll);
    display(tile_ur)
    for i in range(4):
        threadId = i
        thParams = threadId, blockId_x, blockId_y 
        A = transpose_enplace(A, N, thParams, tile_ur, tile_ll);
    display(A)
    #display(tile_ll)

In [ ]:
np.array_equal(A, AT)